<a href="https://colab.research.google.com/github/mahshidkhatiri/MachineLearning/blob/main/auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import urllib
import numpy as np
from sklearn.preprocessing import StandardScaler
import IPython
!pip install -q -U keras-tuner
import keras_tuner as kt
warnings.filterwarnings('ignore')
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
file = urllib.request.urlopen(url)
x=[]
for line in file:
    decoded_line = line.decode("utf-8")
    [values,car_name] = decoded_line.strip().split("\t")
    values=values.split()
    values.append(car_name)
    x.append(values)
df = pd.DataFrame(x,
               columns =['mpg','cylinders','displacement','horsepower','weight'
                            ,'acceleration','model_year','origin','car_name'])
df=df.drop(['origin','car_name'], axis=1)
df.model_year='19'+df.model_year
df1 = df.apply(pd.to_numeric, errors='coerce')
mean_horsepower= df1['horsepower'].mean(axis = 0, skipna = True)
df1['horsepower']=df1['horsepower'].replace(np.nan,mean_horsepower)
train, validation = train_test_split(df1, test_size=0.5, random_state=42)
train_labels = train.iloc[:,0]
val_labels = validation.iloc[:,0]
x_train=train.drop(['mpg'], axis=1)
x_valid=validation.drop(['mpg'], axis=1)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_valid = scaler.transform(x_valid)


In [9]:
def model_builder(hp):
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=[6]))
  model.add(keras.layers.BatchNormalization())

  hp_units_1 = hp.Int('units_1', min_value = 10, max_value = 80, step = 4)
  model.add(keras.layers.Dense(units = hp_units_1, activation="elu"))
  model.add(keras.layers.BatchNormalization())
  model. add(keras.layers.Dropout(rate=0.5))
  hp_units_2 = hp.Int('units_2', min_value = 0, max_value = 100, step = 5)
  model.add(keras.layers.Dense(units = hp_units_2,activation="relu" ,kernel_initializer="he_normal"))
  model. add(keras.layers.Dropout(rate=0.5))
  model.add(keras.layers.BatchNormalization())
  hp_units_3 = hp.Int('units_3', min_value = 0, max_value = 100, step = 5)
  model.add(keras.layers.Dense(units = hp_units_3, activation="relu"))
  model. add(keras.layers.Dropout(rate=0.5))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dense(1, activation='relu'))

  # Tune the optimizer:
  hp_optimizer = hp.Choice('optimizer', ['Adam', 'RMSProp', 'SGD', 'Adagrad', 'Adamax'])
  model.compile(optimizer = hp_optimizer,
                loss = keras.losses.MeanSquaredError())
  return model

In [10]:
tuner = kt.Hyperband(model_builder,
                      objective = kt.Objective('val_loss', direction='min'),
                     max_epochs = 20,
                     factor = 3,
                     overwrite = True)


In [ ]:
tuner.search(x_train, train_labels, epochs = 20, validation_data=(x_valid, val_labels))

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print(best_hps.values)
model = tuner.hypermodel.build(best_hps)
keras.utils.plot_model(model, "auto.png", show_shapes=True, rankdir='LR')

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        print(logs)
        if(logs.get('val_loss')<7):
            print("Reached good training!")
            self.model.stop_training = True
callback = myCallback()
history=model.fit(x_train, train_labels, epochs=100,validation_data=(x_valid, val_labels),callbacks=[callback], verbose=1)


In [ ]:
x_test = np.array([[6, 2170, 502, 3164, 4.2, 2023], #predicted:"34.7"
[12, 6498 ,730 ,3472, 3.2, 2023], #predicted:"24.4"
 [8 ,3902 ,986 ,3020 ,2.5, 2023],#predicted:"19.5"
 [8 ,6162 ,670 ,3721 ,2.6 ,2023],#predicted:"24.6"
[4 ,122, 181 ,2496, 8.3 ,2023],#predicted:"50.9"
[6 ,3232, 155 ,3232, 11.5, 1969],#predicted:"16.5"
[3 ,598, 89, 1550 ,10.1, 2023],#predicted:"60"
[3 ,900, 50, 642, 5.8, 2023],#predicted:"67"
[4 ,1189 ,60 ,2355 ,28.1 ,1964],#predicted:"23"
[4 ,201, 40 ,2265 ,32, 1908]])#predicted:"8"
x_test_s= scaler.transform(x_test)
y_test= model.predict(x_test_s)
for i in range(len(x_test)):
  print("X=%s, Predicted mpg=%s" % (x_test[i], y_test[i]))
  """As far as I searched about cars I understood sport cars' mpg is around 25 since the fuel efficiency doesn't matter in them
   also it is because they are heavy and they have high horsepower also the displacement is large so these are the cause of the
    low mpg in them."""

